In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score

from sklearn.model_selection import KFold

from sklearn.model_selection import GridSearchCV  #Perforing grid search

import re 

In [2]:
train_limpio = pd.read_csv("train_limpio_con_BOW_de_5000_y_Stemming_2.csv",encoding = "ISO-8859-1")
train_limpio.head(2)

,aa,ab,aba,abandon,abbott,abbswinston,abc,abcnew,abe,abil,...,wildfir-keyword,windstorm-keyword,wound-keyword,wreck-keyword,wreckag-keyword,isRealPlace,longitud_tweet,letras_seguidas,sentimiento,objetividad
0,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,70,2,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,40,1,0.1,0.4


In [16]:
test_limpio = pd.read_csv("test_limpio_con_BOW_de_5000_y_Stemming_2.csv",encoding = "ISO-8859-1")
test_limpio.head(2)

,aa,ab,aba,abandon,abbott,abbswinston,abc,abcnew,abe,abil,...,wildfir-keyword,windstorm-keyword,wound-keyword,wreck-keyword,wreckag-keyword,isRealPlace,longitud_tweet,letras_seguidas,sentimiento,objetividad
0,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,40,2,-1.00,1.00
1,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,70,2,0.25,0.55


In [4]:
sample = pd.read_csv("sample_submission.csv",encoding = "ISO-8859-1")
sample.head(2)

,id,target
0,0,0
1,2,0


In [3]:
df_x=train_limpio.drop(['target'], axis=1)
df_x.head()

,aa,ab,aba,abandon,abbott,abbswinston,abc,abcnew,abe,abil,...,wildfir-keyword,windstorm-keyword,wound-keyword,wreck-keyword,wreckag-keyword,isRealPlace,longitud_tweet,letras_seguidas,sentimiento,objetividad
0,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,70,2,0.00000,0.0000
1,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,40,1,0.10000,0.4000
2,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,130,2,-0.01875,0.3875
3,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,60,1,0.00000,0.0000
4,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,90,2,0.00000,0.0000


In [4]:
df_y=train_limpio.target
df_y

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [8]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=4)

In [7]:
gbl = GradientBoostingClassifier()
gbl.fit(x_train, y_train)

GradientBoostingClassifier()

In [8]:
pred = gbl.predict(x_test)
pred

array([1, 0, 0, ..., 0, 0, 0])

In [9]:
f1_score(y_test.values, pred, average='macro')

0.7369431019449149

# Tunning con grid search

In [7]:
from sklearn.metrics import f1_score, make_scorer

#Se tendrian que usar estos parametros, pero todos juntos no termino nunca
    #'n_estimators':[120,140,160,180],
    #'learning_rate': [0.15, 0.2, 0.3]
    #"min_samples_split": np.linspace(0.1, 0.5, 6)
    #"min_samples_leaf": np.linspace(0.1, 0.5, 6),
parameters = {
    'n_estimators':[110,120],
    'learning_rate': [0.15, 0.2]
}
clf = GridSearchCV(GradientBoostingClassifier(), parameters,cv=5,n_jobs=-1,scoring='f1_micro')
clf.fit(df_x, df_y)
#print(clf.score(x_train, y_train))
print(clf.best_params_)

{'learning_rate': 0.15, 'n_estimators': 110}


In [6]:
print(clf.best_params_)

{'learning_rate': 0.15, 'n_estimators': 110}


# Hago k-folds con los mejores parametros obtenidos del modelo anterior

In [10]:
kf = KFold(n_splits = 5)
kf

KFold(n_splits=5, random_state=None, shuffle=False)

In [13]:
#Uso los parametros del modelo anterior
for train_index,test_index in kf.split(train_limpio):
    x_train,x_test = df_x.loc[train_index], df_x.loc[test_index]
    y_train,y_test = df_y[train_index], df_y[test_index]
    
    gbl = GradientBoostingClassifier(learning_rate= 0.48,n_estimators= 160,min_samples_split=12,min_samples_leaf=1)
    gbl.fit(x_train, y_train)
    
    #error de entrenamiento:
    pred = gbl.predict(x_train)
    print('Error de entrenamiento: ',f1_score(y_train.values, pred, average='macro'))
    
    
        
    #error de test:
    pred = gbl.predict(x_test)
    print('Error de test: ',f1_score(y_test.values, pred, average='macro'))

Error de entrenamiento:  0.896548163609717
Error de test:  0.7239576384056776
Error de entrenamiento:  0.9044179680181123
Error de test:  0.7243759704466495
Error de entrenamiento:  0.9057547122643868
Error de test:  0.692567865365833
Error de entrenamiento:  0.9005081067422044
Error de test:  0.7206399919440294
Error de entrenamiento:  0.8948541227069801
Error de test:  0.7545294710009134


# Analizo sus mejores features y los remuevo con SHAP (esto hacerlo despues del tunning con gridd search)

# Realizo la prediccion

In [17]:
# Entonces, mi modelo optimizado es:
train_x=df_x

train_y=df_y



gbl = GradientBoostingClassifier(learning_rate= 0.48,n_estimators= 160,min_samples_split=12,min_samples_leaf=1)
gbl.fit(x_train, y_train)

    
#error de entrenamiento, es el unico que puedo calcular:
pred = gbl.predict(train_x)
print('Error de entrenamiento: ',f1_score(train_y, pred, average='macro'))


Error de entrenamiento:  0.8657030579252265


In [18]:
pred = gbl.predict(test_limpio)

In [19]:
sample.target=pred

In [20]:
sample.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [21]:
sample.to_csv('prediccion_gradient_boosting.csv', index=False)

In [ ]:
#https://www.kaggle.com/hatone/gradientboostingclassifier-with-gridsearchcv
#https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/